In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DATA_PATH = '/home/ataleckij/Projects/university/mo/Data/Lab 4/'

In [3]:
files_names = os.listdir(DATA_PATH)
print(files_names)
files_paths = [os.path.join(DATA_PATH, name) for name in files_names]

['ex4data1.mat', 'ex4weights.mat']


In [7]:
from scipy.io import loadmat

data = loadmat(files_paths[0])
weights = loadmat(files_paths[1])

features, labels = data['X'], data['y'].flatten()
t1, t2 = weights['Theta1'], weights['Theta2']
print(features.shape, labels.shape)
print(t1.shape, t2.shape)

(5000, 400) (5000,)
(25, 401) (10, 26)


In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels_indexes = le.fit_transform(labels)

labels_onehot = np.zeros((labels.shape[0], labels_indexes.max() + 1))
labels_onehot[np.arange(labels.shape[0]), labels_indexes] = 1

print(labels[:7])
print(labels_onehot[:7])

[10 10 10 10 10 10 10]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [9]:
w1, b1 = t1[:, 1:], t1[:, [0]]
w2, b2 = t2[:, 1:], t2[:, [0]]

In [10]:
import numpy as np
import random

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

# def cost_function(network, test_data):
#     features, labels_onehot = test_data
    
#     c = 0
#     for example, y_onehot in test_data:
#         prediction_onehot = network.feedforward(example)
#         c += np.sum((y_onehot - prediction_onehot)**2)
#     return c / len(test_data)

def cost_function(prediction_onehot, y_onehot, w, lambda_):
    return (np.sum(y_onehot, -prediction_onehot, axis=1) ** 2).mean() + 1/2*lambda_ * w.dot(w)

In [94]:
class Network:
    def __init__(self, sizes, output=True):
        """
        Список ``sizes`` содержит количество нейронов в соответствующих слоях
        нейронной сети. Смещения и веса для нейронных сетей
        инициализируются случайными значениями, подчиняющимися стандартному нормальному
        распределению. Первый слой подразумевается слоем, принимающим входные данные, 
        поэтому к нему не добавляется смещение
        """

        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.output = output

    def feedforward(self, a):
        """
        Вычислить и вернуть выходную активацию нейронной сети
        при получении ``a`` на входе
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b.T[0])
        return a
    
    def predict(self, features):
        return np.asarray([self.feedforward(example_features) 
                           for example_features in features])
    
    def predict_classes(self, features):
        prediction_onehot = self.predict(features)
        return np.argmax(prediction_onehot, axis=1)

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """
        Обучить нейронную сеть, используя алгоритм стохастического
        (mini-batch) градиентного спуска. 
        ``training_data`` - лист кортежей вида ``(x, y)``, где 
        x - вход обучающего примера, y - желаемый выход (в формате one-hot). 
        Роль остальных обязательных параметров должна быть понятна из их названия.
        Если предоставлен опциональный аргумент ``test_data``, 
        то после каждой эпохи обучения сеть будет протестирована на этих данных 
        и промежуточный результат обучения будет выведен в консоль. 
        ``test_data`` -- это список кортежей из входных данных 
        и номеров правильных классов примеров (т.е. argmax(y),
        если y -- набор ответов в той же форме, что и в тренировочных данных).
        Тестирование полезно для мониторинга процесса обучения,
        но может существенно замедлить работу программы.
        """

        if test_data is not None: n_test = len(test_data)
        n = len(training_data)
        success_tests = 0
        training_data = [(x[:, np.newaxis], y[:, np.newaxis]) for x, y in training_data]
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size]
                            for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data is not None and self.output:
                success_tests = self.evaluate(test_data)
                print("Эпоха {0}: {1} / {2}".format(j, success_tests, n_test))
            elif self.output:
                print("Эпоха {0} завершена".format(j))
        if test_data is not None:
            return success_tests / n_test

    def update_mini_batch(self, mini_batch, eta):
        """
        Обновить веса и смещения нейронной сети, сделав шаг градиентного
        спуска на основе алгоритма обратного распространения ошибки, примененного
        к одному mini batch.
        ``mini_batch`` - список кортежей вида ``(x, y)``,
        ``eta`` - величина шага (learning rate).
        """
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        eps = eta / len(mini_batch)
        self.weights = [w - eps * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases  = [b - eps * nb for b, nb in zip(self.biases,  nabla_b)]

    def backprop(self, x, y):
        """
        Возвращает кортеж ``(nabla_b, nabla_w)`` -- градиент целевой функции по всем параметрам сети.
        ``nabla_b`` и ``nabla_w`` -- послойные списки массивов ndarray,
        такие же, как self.biases и self.weights соответственно.
        """

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # прямое распространение (forward pass)

        a = [np.array(x).copy()]
        z = []
        for b, w in zip(self.biases, self.weights):
            # посчитать активации
            if len(a) > 1:
                zi = np.dot(w, a[-1]) + b#.T[0]
            else:
                zi = np.dot(w, a[-1])# + b#.T[0]
            ai = sigmoid(zi)
            z.append(zi)
            a.append(ai)
        z = np.array(z)
        a = np.array(a)

        # обратное распространение (backward pass)
        delta = (a[-1] - y)*sigmoid_prime(z[-1])#/(a[-1]*(1-a[-1]))  # ошибка выходного слоя
        nabla_b[-1] = delta # производная J по смещениям выходного слоя
        nabla_w[-1] = np.dot(delta, a[-2].T)  # производная J по весам выходного слоя

        for l in range(2, self.num_layers):
            # дополнительные вычисления, чтобы легче записывалось
            #
            weights = np.array(self.weights).copy()
            delta = np.dot(weights[-(l-1)].T, delta)*sigmoid_prime(z[-l]) # ошибка на слое L-l
            nabla_b[-l] = delta # производная J по смещениям L-l-го слоя
            nabla_w[-l] = np.dot(delta, a[-(l+1)].T) # производная J по весам L-l-го слоя
        return nabla_b, nabla_w

    def evaluate(self, test_data):
        """
        Вернуть количество тестовых примеров, для которых нейронная сеть
        возвращает правильный ответ.
        """
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """
        Возвращает вектор частных производных целевой функции по активациям выходного слоя.
        """
        return (output_activations-y)

In [62]:
nn = Network([400, 25, 10])

print([b.shape for b in nn.biases])
print([w.shape for w in nn.weights])

print(b1.shape, b2.shape)
print(w1.shape, w2.shape)

nn.biases = [b1, b2]
nn.weights = [w1, w2]

[(25, 1), (10, 1)]
[(25, 400), (10, 25)]
(25, 1) (10, 1)
(25, 400) (10, 25)


In [63]:
predicted_indexes = nn.predict_classes(features)
predicted_labels = le.inverse_transform(predicted_indexes)

(predicted_labels == labels).mean()

0.9752

In [95]:
class RegularizedNetwork(Network):
    def __init__(self, sizes, output=True, l2=0):
        super().__init__(sizes, output)
        self.l2 = l2
        
    def update_mini_batch(self, mini_batch, eta):
        """
        Обновить веса и смещения нейронной сети, сделав шаг градиентного
        спуска на основе алгоритма обратного распространения ошибки, примененного
        к одному mini batch. Учесть штраф L2.
        ``mini_batch`` - список кортежей вида ``(x, y)``,
        ``eta`` - величина шага (learning rate).
        """
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        eps = eta / len(mini_batch)
        self.weights = [w - eps * nw - self.l2 * w for w, nw in zip(self.weights, nabla_w)]
        self.biases  = [b - eps * nb for b, nb in zip(self.biases,  nabla_b)]

In [96]:
nn_l2 = RegularizedNetwork([400, 25, 10], l2=0.00001)

train_data = list(zip(features, labels_onehot))
nn_l2.SGD(train_data, eta=0.5, mini_batch_size=16, epochs=20)

Эпоха 0 завершена
Эпоха 1 завершена
Эпоха 2 завершена
Эпоха 3 завершена
Эпоха 4 завершена
Эпоха 5 завершена
Эпоха 6 завершена
Эпоха 7 завершена
Эпоха 8 завершена
Эпоха 9 завершена
Эпоха 10 завершена
Эпоха 11 завершена
Эпоха 12 завершена
Эпоха 13 завершена
Эпоха 14 завершена
Эпоха 15 завершена
Эпоха 16 завершена
Эпоха 17 завершена
Эпоха 18 завершена
Эпоха 19 завершена


In [97]:
predicted_indexes = nn_l2.predict_classes(features)
predicted_labels = le.inverse_transform(predicted_indexes)

(predicted_labels == labels).mean()

0.8146